In [1]:
import scipy.linalg
import scipy.optimize
import sys
import numpy as np
import importlib
import matplotlib.pyplot as plt
import scripts.theodolite_function as tfu
tfu = importlib.reload(tfu)
from scipy.interpolate import splprep, splev
from scipy import interpolate
import seaborn as sns
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from scipy.spatial.transform import Rotation as R
import scripts.theodolite_utils as tu
import scripts.resection_functions as rf


Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


In [2]:
# 17 02 2022 / 24 02 2022
Dist_prism_12_240222 = 0.8882613537922258
Dist_prism_13_240222 = 0.8262903370535766
Dist_prism_23_240222 = 1.0377436460567973
Dist_GPS_12_240222 = 0.7809786915630756
Dist_GPS_13_240222 = 0.8378323585433202
Dist_GPS_23_240222 = 0.5191788936274444   

# 07 03 2022
Dist_prism_12_070322 = 0.8832836583456264
Dist_prism_13_070322 = 0.8247449387008166
Dist_prism_23_070322 = 1.039409793600046
Dist_GPS_12_070322 = 0.7781099344563802
Dist_GPS_13_070322 = 0.8431026980109139
Dist_GPS_23_070322 = 0.5157221841320069

# 12 03 2022
Dist_prism_12_120322 = 0.8860734460339026
Dist_prism_13_120322 = 0.8303896290142598
Dist_prism_23_120322 = 1.0393117663247973
Dist_GPS_12_120322 = 0.7829424296161074
Dist_GPS_13_120322 = 0.8415176433032493
Dist_GPS_23_120322 = 0.5161917038941966

# 14 03 2022 / 16 03 2022
Dist_prism_12_140322 = 0.8873171656784946
Dist_prism_13_140322 = 0.8272212117473343
Dist_prism_23_140322 = 1.0379270641796363
Dist_GPS_12_140322 = 0.7829081036179948
Dist_GPS_13_140322 = 0.8424633975958637
Dist_GPS_23_140322 = 0.5165807364575292

# 31 03 2022
Dist_prism_12_310322 = 0.8873171656784946
Dist_prism_13_310322 = 0.8272212117473343
Dist_prism_23_310322 = 1.0379270641796363
Dist_GPS_12_310322 = 0.7829081036179948
Dist_GPS_13_310322 = 0.8424633975958637
Dist_GPS_23_310322 = 0.5165807364575292

# 27 04 2022
Dist_prism_12_270422 = 0.8856608085851714
Dist_prism_13_270422 = 0.8264452483880412
Dist_prism_23_270422 = 1.0384219210289538
Dist_GPS_12_270422 = 0.7809547832390561
Dist_GPS_13_270422 = 0.8259590286006459
Dist_GPS_23_270422 = 0.5191274645630007  

# 27 04 2022 soir
Dist_prism_12_270422_soir = 0.8868846942534616
Dist_prism_13_270422_soir = 0.829169129877633
Dist_prism_23_270422_soir = 1.0387571594276301
Dist_GPS_12_270422_soir = 0.7837749633840095
Dist_GPS_13_270422_soir = 0.8323125081688734
Dist_GPS_23_270422_soir = 0.5187055120945835

# 05 05 2022
Dist_prism_12_050522 = 0.3819811991689936
Dist_prism_13_050522 = 0.4426382054042266
Dist_prism_23_050522 = 0.2564685508415531

# 13 05 2022
Dist_prism_12_130522 = 0.8113569618671205
Dist_prism_13_130522 = 0.8695313906832193
Dist_prism_23_130522 = 1.0383292631343506
Dist_GPS_12_130522 = 0.7800801192552722
Dist_GPS_13_130522 = 0.8396027021467387
Dist_GPS_23_130522 = 0.5157656903777014

# 23 05 2022
Dist_prism_12_230522 = 0.3851913749758221
Dist_prism_13_230522 = 0.4433899497583272
Dist_prism_23_230522 = 0.25861327466684897

# 25 05 2022
Dist_prism_12_250522 = 0.9095421527752512
Dist_prism_13_250522 = 0.7356677023921305
Dist_prism_23_250522 = 1.0283049452358466

# 22 06 2022
Dist_prism_12_220622 = 0.7359204697527427
Dist_prism_13_220622 = 0.9055056357941131
Dist_prism_23_220622 = 1.0268390664025775

# 30 06 2022
Dist_prism_12_300622 = 0.7297583705742492
Dist_prism_13_300622 = 0.9060320633355149
Dist_prism_23_300622 = 1.0319114860615324

# 11 07 2022
Dist_prism_12_110722 = 0.7290056530579362
Dist_prism_13_110722 = 0.906995998154099
Dist_prism_23_110722 = 1.029264651249632

# 17 07 2022
Dist_prism_12_170722 = 0.39002864329744197 
Dist_prism_13_170722 = 0.4452628757931205 
Dist_prism_23_170722 = 0.2597453130254009

# 10 09 2022
Dist_prism_12_100922 = 0.38166035383298946
Dist_prism_13_100922 = 0.4321757774250811 
Dist_prism_23_100922 = 0.26468652985485586

In [3]:
def check_if_converge_well(arr_error, Tf_results, debug):
    
    sorted_arr_error = np.argsort(arr_error)
    minimum_error_index = sorted_arr_error[0]
    TF_1_ref = Tf_results[minimum_error_index][0]
    TF_2_ref = Tf_results[minimum_error_index][1]
    r1 = R.from_matrix(TF_1_ref[0:3,0:3])
    r2 = R.from_matrix(TF_2_ref[0:3,0:3])
    euler1_ref = r1.as_euler('xyz', degrees=True)
    euler2_ref = r2.as_euler('xyz', degrees=True)

    close_enough = False
    for error_sorted in sorted_arr_error[1:3]:
        TF_1 = Tf_results[error_sorted][0]
        TF_2 = Tf_results[error_sorted][1]
        r1 = R.from_matrix(TF_1[0:3,0:3])
        r2 = R.from_matrix(TF_2[0:3,0:3])
        euler1 = r1.as_euler('xyz', degrees=True)
        euler2 = r2.as_euler('xyz', degrees=True)
        r1_diff = abs(euler1[2]-euler1_ref[2])
        r2_diff = abs(euler2[2]-euler2_ref[2])
        t1_diff = np.linalg.norm(TF_1[0:3,3]-TF_1_ref[0:3,3])
        t2_diff = np.linalg.norm(TF_2[0:3,3]-TF_2_ref[0:3,3])
        if(debug):
            print("Index: ", error_sorted)
            print("Diff: ",r1_diff,r2_diff,t1_diff,t2_diff)
        if(r1_diff>0.5 or r2_diff>0.5 or t1_diff>0.05 or t2_diff>0.05):
            close_enough = False
            break
        else:
            close_enough = True

    if(close_enough==False):
        minimum_error_index = sorted_arr_error[1]
        TF_1_ref = Tf_results[minimum_error_index][0]
        TF_2_ref = Tf_results[minimum_error_index][1]
        r1 = R.from_matrix(TF_1_ref[0:3,0:3])
        r2 = R.from_matrix(TF_2_ref[0:3,0:3])
        euler1_ref = r1.as_euler('xyz', degrees=True)
        euler2_ref = r2.as_euler('xyz', degrees=True)

        close_enough = False
        for error_sorted in sorted_arr_error[2:4]:
            TF_1 = Tf_results[error_sorted][0]
            TF_2 = Tf_results[error_sorted][1]
            r1 = R.from_matrix(TF_1[0:3,0:3])
            r2 = R.from_matrix(TF_2[0:3,0:3])
            euler1 = r1.as_euler('xyz', degrees=True)
            euler2 = r2.as_euler('xyz', degrees=True)
            r1_diff = abs(euler1[2]-euler1_ref[2])
            r2_diff = abs(euler2[2]-euler2_ref[2])
            t1_diff = np.linalg.norm(TF_1[0:3,3]-TF_1_ref[0:3,3])
            t2_diff = np.linalg.norm(TF_2[0:3,3]-TF_2_ref[0:3,3])
            if(debug):
                print("Index: ", error_sorted)
                print("Diff: ",r1_diff,r2_diff,t1_diff,t2_diff)
            if(r1_diff>0.5 or r2_diff>0.5 or t1_diff>0.05 or t2_diff>0.05):
                close_enough = False
                break
            else:
                close_enough = True

    if(close_enough==False):
        minimum_error_index = sorted_arr_error[2]
        TF_1_ref = Tf_results[minimum_error_index][0]
        TF_2_ref = Tf_results[minimum_error_index][1]
        r1 = R.from_matrix(TF_1_ref[0:3,0:3])
        r2 = R.from_matrix(TF_2_ref[0:3,0:3])
        euler1_ref = r1.as_euler('xyz', degrees=True)
        euler2_ref = r2.as_euler('xyz', degrees=True)

        close_enough = False
        for error_sorted in sorted_arr_error[3:5]:
            TF_1 = Tf_results[error_sorted][0]
            TF_2 = Tf_results[error_sorted][1]
            r1 = R.from_matrix(TF_1[0:3,0:3])
            r2 = R.from_matrix(TF_2[0:3,0:3])
            euler1 = r1.as_euler('xyz', degrees=True)
            euler2 = r2.as_euler('xyz', degrees=True)
            r1_diff = abs(euler1[2]-euler1_ref[2])
            r2_diff = abs(euler2[2]-euler2_ref[2])
            t1_diff = np.linalg.norm(TF_1[0:3,3]-TF_1_ref[0:3,3])
            t2_diff = np.linalg.norm(TF_2[0:3,3]-TF_2_ref[0:3,3])
            if(debug):
                print("Index: ", error_sorted)
                print("Diff: ",r1_diff,r2_diff,t1_diff,t2_diff)
            if(r1_diff>0.5 or r2_diff>0.5 or t1_diff>0.05 or t2_diff>0.05):
                close_enough = False
                break
            else:
                close_enough = True

    if(close_enough==True):  
        print("Index minimum error: ", minimum_error_index)
        print("Speed: ", l[minimum_error_index])
        print("Minimum error: ", arr_error[minimum_error_index])
        return minimum_error_index
    else:
        print("Not converged well enough !! ")
        return -1

In [4]:
# This cell gathers result for the inter-prism distance resection for the graphic below
# With a linear interpolation, and 3 CP only

rf = importlib.reload(rf)

name_param_file = 'f-2-1-1-1-6-0-L_'

linear_inter_prism_file_name = [
                                './data/prediction/TS/20220224/filtered/'+name_param_file,    # bad
                                './data/prediction/TS/20220307/filtered/'+name_param_file,    # okay
                                './data/prediction/TS/20220314/filtered/'+name_param_file,    # okay
                                './data/prediction/TS/20220316/filtered/'+name_param_file,    # not great
                                './data/prediction/TS/20220331-2/filtered/'+name_param_file,  # okay
                                './data/prediction/TS/20220513/01/filtered/'+name_param_file,   # not bad!
                                './data/prediction/TS/20220513/02/filtered/'+name_param_file,   # badder
                                './data/prediction/TS/20220513/03/filtered/'+name_param_file,   # not great
                                './data/prediction/TS/20220513/04/filtered/'+name_param_file,   # best one yet
                                './data/prediction/TS/20220513/06/filtered/'+name_param_file,   # okay
                                './data/prediction/TS/20220525/01/filtered/'+name_param_file,
                                './data/prediction/TS/20220525/02/filtered/'+name_param_file,
                                './data/prediction/TS/20220622-1/filtered/'+name_param_file,
                                './data/prediction/TS/20220622-2/filtered/'+name_param_file,
                                './data/prediction/TS/20220630-1/filtered/'+name_param_file,
                                './data/prediction/TS/20220630-2/filtered/'+name_param_file,    # okay
                                './data/prediction/TS/20220711/02/filtered/'+name_param_file   # bad
    
#                                 './data/prediction/TS/20220331-1/filtered/'+name_param_file,  # bad
#                                 './data/prediction/TS/20220505_cones/filtered/'+name_param_file,  # badder
#                                 './data/prediction/TS/20220505_empty/filtered/'+name_param_file,  # badder
#                                 './data/prediction/TS/20220523_cones/filtered/'+name_param_file,
#                                 './data/prediction/TS/20220523_constrained/filtered/'+name_param_file,
#                                 './data/prediction/TS/20220523_empty/filtered/'+name_param_file,
#                                 './data/prediction/TS/20220523_other_tunnel/filtered/'+name_param_file,
#                                 './data/prediction/TS/20220717/02/filtered/'+name_param_file,
#                                 './data/prediction/TS/20220717/03/filtered/'+name_param_file,
#                                 './data/prediction/TS/20220717/04/filtered/'+name_param_file
    
#                                 './data/prediction/TS/20220910/07/filtered/'+name_param_file,
#                                 './data/prediction/TS/20220910/08/filtered/'+name_param_file,
#                                 './data/prediction/TS/20220910/09/filtered/'+name_param_file,
#                                 './data/prediction/TS/20220910/10/filtered/'+name_param_file
                               ]
file_name_marker = [
                    './data/total_station/20220224/20220224_theodolite_reference_prisms.txt',
                    './data/total_station/20220307/theodolite_reference_prisms.txt',
                    './data/total_station/20220314/theodolite_reference_prisms.txt',
                    './data/total_station/20220316/theodolite_reference_prisms.txt',
                    './data/total_station/20220331-2/theodolite_reference_prisms.txt',
                    './data/total_station/20220513_morning_lidar/theodolite_reference_prisms.txt',
                    './data/total_station/20220513_morning_without_lidar_1/theodolite_reference_prisms.txt',
                    './data/total_station/20220513_morning_without_lidar_2/theodolite_reference_prisms.txt',
                    './data/total_station/20220513_evening_lidar/theodolite_reference_prisms.txt',
                    './data/total_station/20220513_evening_without_lidar_2/theodolite_reference_prisms.txt',
                    './data/total_station/20220525/theodolite_reference_prisms.txt',
                    './data/total_station/20220525/theodolite_reference_prisms.txt',
                    './data/total_station/20220622-1/theodolite_reference_prisms_2.txt',
                    './data/total_station/20220622-2/theodolite_reference_prisms_2.txt',
                    './data/total_station/20220630-1/theodolite_reference_prisms_without_first_two.txt',
                    './data/total_station/20220630-2/theodolite_reference_prisms_without_first_two.txt',
                    './data/total_station/20220711/theodolite_reference_prisms_1_to_10.txt'
    
#                     './data/total_station/20220331-1/theodolite_reference_prisms.txt',
#                     './data/total_station/20220505_cones/theodolite_reference_prisms.txt',
#                     './data/total_station/20220505_empty/theodolite_reference_prisms.txt',
#                     './data/total_station/20220523_cones/theodolite_reference_prisms.txt',
#                     './data/total_station/20220523_constrained/theodolite_reference_prisms.txt',
#                     './data/total_station/20220523_empty/theodolite_reference_prisms.txt',
#                     './data/total_station/20220523_other_tunnel/theodolite_reference_prisms.txt',
#                     './data/total_station/20220717/theodolite_reference_prisms.txt',
#                     './data/total_station/20220717/theodolite_reference_prisms.txt',
#                     './data/total_station/20220717/theodolite_reference_prisms.txt'
    
#                     './data/total_station/20220910/theodolite_reference_prisms_all.txt',
#                     './data/total_station/20220910/theodolite_reference_prisms_all.txt',
#                     './data/total_station/20220910/theodolite_reference_prisms_all.txt',
#                     './data/total_station/20220910/theodolite_reference_prisms_all.txt'
    
                   ]

Inter_distance = [
                  [Dist_prism_12_240222,Dist_prism_13_240222,Dist_prism_23_240222,Dist_GPS_12_240222,Dist_GPS_13_240222,Dist_GPS_23_240222],
                  [Dist_prism_12_070322,Dist_prism_13_070322,Dist_prism_23_070322,Dist_GPS_12_070322,Dist_GPS_13_070322,Dist_GPS_23_070322],
                  [Dist_prism_12_140322,Dist_prism_13_140322,Dist_prism_23_140322,Dist_GPS_12_140322,Dist_GPS_13_140322,Dist_GPS_23_140322],
                  [Dist_prism_12_140322,Dist_prism_13_140322,Dist_prism_23_140322,Dist_GPS_12_140322,Dist_GPS_13_140322,Dist_GPS_23_140322],
                  [Dist_prism_12_310322,Dist_prism_13_310322,Dist_prism_23_310322,Dist_GPS_12_310322,Dist_GPS_13_310322,Dist_GPS_23_310322],
                  [Dist_prism_12_130522,Dist_prism_13_130522,Dist_prism_23_130522,Dist_GPS_12_130522,Dist_GPS_13_130522,Dist_GPS_23_130522],
                  [Dist_prism_12_130522,Dist_prism_13_130522,Dist_prism_23_130522,Dist_GPS_12_130522,Dist_GPS_13_130522,Dist_GPS_23_130522],
                  [Dist_prism_12_130522,Dist_prism_13_130522,Dist_prism_23_130522,Dist_GPS_12_130522,Dist_GPS_13_130522,Dist_GPS_23_130522],
                  [Dist_prism_12_130522,Dist_prism_13_130522,Dist_prism_23_130522,Dist_GPS_12_130522,Dist_GPS_13_130522,Dist_GPS_23_130522],
                  [Dist_prism_12_130522,Dist_prism_13_130522,Dist_prism_23_130522,Dist_GPS_12_130522,Dist_GPS_13_130522,Dist_GPS_23_130522],               
                  [Dist_prism_12_250522,Dist_prism_13_250522,Dist_prism_23_250522],
                  [Dist_prism_12_250522,Dist_prism_13_250522,Dist_prism_23_250522],
                  [Dist_prism_12_220622,Dist_prism_13_220622,Dist_prism_23_220622],
                  [Dist_prism_12_220622,Dist_prism_13_220622,Dist_prism_23_220622],
                  [Dist_prism_12_300622,Dist_prism_13_300622,Dist_prism_23_300622],
                  [Dist_prism_12_300622,Dist_prism_13_300622,Dist_prism_23_300622],
                  [Dist_prism_12_110722,Dist_prism_13_110722,Dist_prism_23_110722]
    
#                   [Dist_prism_12_310322,Dist_prism_13_310322,Dist_prism_23_310322,Dist_GPS_12_310322,Dist_GPS_13_310322,Dist_GPS_23_310322],
#                   [Dist_prism_12_050522,Dist_prism_13_050522,Dist_prism_23_050522],
#                   [Dist_prism_12_050522,Dist_prism_13_050522,Dist_prism_23_050522],
#                   [Dist_prism_12_230522,Dist_prism_13_230522,Dist_prism_23_230522],
#                   [Dist_prism_12_230522,Dist_prism_13_230522,Dist_prism_23_230522],
#                   [Dist_prism_12_230522,Dist_prism_13_230522,Dist_prism_23_230522],
#                   [Dist_prism_12_230522,Dist_prism_13_230522,Dist_prism_23_230522],
#                   [Dist_prism_12_170722,Dist_prism_13_170722,Dist_prism_23_170722],
#                   [Dist_prism_12_170722,Dist_prism_13_170722,Dist_prism_23_170722],
#                   [Dist_prism_12_170722,Dist_prism_13_170722,Dist_prism_23_170722]
    
#                     [Dist_prism_12_100922,Dist_prism_13_100922,Dist_prism_23_100922],
#                     [Dist_prism_12_100922,Dist_prism_13_100922,Dist_prism_23_100922],
#                     [Dist_prism_12_100922,Dist_prism_13_100922,Dist_prism_23_100922],
#                     [Dist_prism_12_100922,Dist_prism_13_100922,Dist_prism_23_100922]
    
                 ]

saving_file_path = [
                    '20220224/',
                    '20220307/',
                    '20220314/',
                    '20220316/',
                    '20220331/02/',
                    '20220513/morning_lidar/',
                    '20220513/morning_without_lidar_1/',
                    '20220513/morning_without_lidar_2/',
                    '20220513/evening_lidar/',
                    '20220513/evening_without_lidar_2/',
                    '20220525/01/',
                    '20220525/02/',
                    '20220622/01/',
                    '20220622/02/',
                    '20220630/01/',
                    '20220630/02/',
                    '20220711/02/'
    
#                     '20220331/01/',  
#                     '20220505/cones/',
#                     '20220505/empty/',
#                     '20220523/cones/',
#                     '20220523/constrained/',
#                     '20220523/empty/',
#                     '20220523/other_tunnel/',
#                     '20220717/'
                   ]


# errors_linear_inter_prism_3cp = []
# errors_linear_inter_prism_exp_3cp = []
# errors_linear_cp_exp_3cp = []
# TF_list_linear_inter_prism_3cp = []
# for i,j,k,l,m in zip(linear_inter_prism_file_name,file_name_marker,Inter_distance,velocity_outlier_list, saving_file_path):
    
#     print("dont give up! It: ", i)
    
#     dist_error,_,errors_prism,_, errors_exp, TF_list = rf.one_inter_prism_resection(Inter_distance=k, 
#                             file_name=i, file_name_marker=j, rate=10, prior = "CP", velocity_outlier = l, 
#                             threshold_training = 0.75, number_iteration = 1, threshold_marker=3, min_6dof=True)
#     print(np.median(errors_prism),np.std(errors_prism))
#     errors_linear_inter_prism_exp_3cp += dist_error
#     errors_linear_cp_exp_3cp += errors_prism
#     TF_list_linear_inter_prism_3cp += TF_list
#     errors_linear_inter_prism_3cp += errors_prism


In [5]:
rf = importlib.reload(rf)

index_file = 16
i = linear_inter_prism_file_name[index_file]
j = file_name_marker[index_file]
k = Inter_distance[index_file]
robot = "warthog"
l = [0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2]
# robot = "marmotte"
# l = [0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2]
m = ''
debug = True
save_results = True


Tf_results = []
Error_results = []
Error_results_cp = []
RF = ['','']
print("File: ",i)
for speed in l:
    print("Speed: ", speed)
    dist_error,_,errors_prism,_, errors_exp, TF_list = rf.one_inter_prism_resection_advanced(Inter_distance=k, 
                            file_name=i, file_name_marker=j, RF=RF, robot=robot, rate=10, prior = "A", velocity_outlier = speed, 
                            threshold_training = 0.75, number_iteration = 1, threshold_marker=1, min_6dof=False)
    if(len(dist_error)>1):
        RF1 = TF_list[0][1]
        RF2 = TF_list[0][2]
        r1 = R.from_matrix(RF1[0:3,0:3])
        r2 = R.from_matrix(RF2[0:3,0:3])
        euler1 = r1.as_euler('xyz', degrees=True)
        euler2 = r2.as_euler('xyz', degrees=True)
        if(debug==True):
            print(RF1[0:3,3],RF2[0:3,3])
            print(euler1,euler2)
            print("Cp error: ", np.median(errors_prism), np.std(errors_prism))
            print("Inter-prism error: ", np.median(errors_exp), np.std(errors_exp))
        RF = [RF1,RF2]
        Tf_results.append([TF_list[0][1],TF_list[0][2]])
        Error_results.append(np.median(errors_exp)) 
        Error_results_cp.append(np.median(errors_prism)) 
    else:
        Tf_results.append(['',''])
        Error_results.append(100000) 
        Error_results_cp.append(100000) 

arr_error = np.array(Error_results)
index_error = check_if_converge_well(arr_error, Tf_results, debug)

if(index_error!=-1):

    errors_linear_dynamic_local = []
    errors_linear_dynamic_cp_local = []
    errors_linear_dynamic_cp_exp_local = []
    Tf_results_bis = []

    Error_results_bis = []
    Error_results_cp_bis = []
    RF = Tf_results[index_error]
    print("File: ",i)
    for speed in l:
        print("Speed: ", speed)
        dist_error,_,errors_prism,_, errors_exp, TF_list = rf.one_inter_prism_resection_advanced(Inter_distance=k, 
                                file_name=i, file_name_marker=j, RF=RF, robot=robot, rate=10, prior = "A", velocity_outlier = speed, 
                                threshold_training = 0.75, number_iteration = 1, threshold_marker=1, min_6dof=False)

        errors_linear_dynamic_local.append(dist_error)
        errors_linear_dynamic_cp_local.append(errors_prism)
        errors_linear_dynamic_cp_exp_local.append(errors_exp)

        if(len(dist_error)>1):
            RF1 = TF_list[0][1]
            RF2 = TF_list[0][2]
            r1 = R.from_matrix(RF1[0:3,0:3])
            r2 = R.from_matrix(RF2[0:3,0:3])
            euler1 = r1.as_euler('xyz', degrees=True)
            euler2 = r2.as_euler('xyz', degrees=True)
            if(debug==True):
                print(RF1[0:3,3],RF2[0:3,3])
                print(euler1,euler2)
                print("Cp error: ", np.median(errors_prism), np.std(errors_prism))
                print("Inter-prism error: ", np.median(errors_exp), np.std(errors_exp))
            Tf_results_bis.append([TF_list[0][1],TF_list[0][2]])
            Error_results_bis.append(np.median(errors_exp))
            Error_results_cp_bis.append(np.median(errors_prism))
        else:
            Tf_results_bis.append(['',''])
            Error_results_bis.append(100000) 
            Error_results_cp_bis.append(100000) 

    arr_error_bis = np.array(Error_results_bis)
    index_error_bis = check_if_converge_well(arr_error_bis, Tf_results_bis, debug)
    print("CP error minimum: ", Error_results_cp_bis[index_error_bis])

    Tf_list_minim = []
    T1 = np.identity(4)
    Tf_list_minim.append(T1.tolist())
    for i in Tf_results_bis[index_error_bis]:
        Tf_list_minim.append(i.tolist())

    if(save_results and index_error_bis!=-1):
        tu.save_error_list_to_file(errors_linear_dynamic_cp_local[index_error_bis], saving_file_path[index_file]+"errors_linear_dynamic_cp_4dof")
        tu.save_error_list_to_file(errors_linear_dynamic_cp_exp_local[index_error_bis], saving_file_path[index_file]+"errors_linear_dynamic_cp_experiment_4dof")
        tu.save_tf_list_to_file([Tf_list_minim], saving_file_path[index_file]+"TF_list_linear_dynamic_4dof")


File:  ./data/prediction/TS/20220711/02/filtered/f-2-1-1-1-6-0-L_
Speed:  0.01
Selected points:  20
Time [s]:  0.24053478240966797
Results done !
[ 2.51689988  5.31988011 -0.30485552] [ 0.0173623  -5.44776238 -0.25963388]
[  0.           0.         -29.71952981] [  0.           0.         -26.31189595]
Cp error:  22.645682698606777 12.635869703782582
Inter-prism error:  3.8781570989276926 16.79778015038073
Speed:  0.05
Selected points:  29


/home/norlab/Repo_git/code_publication_ICRA2023_MaximeVaidis/scripts/resection_functions.py:629: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(prior=="PTP" or (prior =="A" and RF[0]=='')):


Time [s]:  0.2249307632446289
Results done !
[ 2.3675529   5.64449955 -0.19441635] [ 0.410645   -5.68329791  0.7724948 ]
[  0.           0.         -29.62149393] [  0.           0.         -26.47945303]
Cp error:  1125.8547915256727 395.08588493291336
Inter-prism error:  123.53127735704983 350.74474658869485
Speed:  0.1
Selected points:  40
Time [s]:  0.2475419044494629
Results done !
[ 2.54676008  5.28593573 -0.27908423] [ 0.0558388  -5.46909122 -0.30994889]
[  0.           0.         -29.73579938] [  0.           0.         -26.32880972]
Cp error:  59.570574435088965 12.639512118684456
Inter-prism error:  7.202259733133154 21.22595502706666
Speed:  0.2
Selected points:  48
Time [s]:  0.24232268333435059
Results done !
[ 2.36503326  5.61018073 -0.2564566 ] [ 0.31677594 -5.54357961  0.64681338]
[  0.           0.         -29.63311393] [  0.           0.         -26.40761303]
Cp error:  965.6963813809006 336.4518232593633
Inter-prism error:  92.99320863320094 274.0043183189096
Speed:  0

Time [s]:  0.3889009952545166
Jump detected !
Time [s]:  0.11793684959411621
Results done !
[ 2.18435094  5.08466374 -0.59498387] [ 1.08855491 -6.98997831  0.14679641]
[  0.           0.         -29.80707565] [  0.           0.         -27.52503738]
Cp error:  1723.120267690052 734.998072716571
Inter-prism error:  504.54108563151146 1084.397701806517
Speed:  0.2
Selected points:  48
Time [s]:  0.2670602798461914
Results done !
[ 2.37901003  5.67310775 -0.21872486] [ 0.31061273 -5.57166222  0.67960863]
[  0.           0.         -29.57754879] [  0.           0.         -26.41609503]
Cp error:  994.1985519722423 330.5662795135493
Inter-prism error:  104.23793508103452 287.31044704982753
Speed:  0.3
Selected points:  70
Time [s]:  0.36323070526123047
Jump detected !
Time [s]:  0.2458968162536621
Results done !
[ 1.81356891  5.01108606 -0.42423763] [ 0.76089348 -5.37744928 -0.36936745]
[  0.           0.         -30.17007667] [  0.           0.         -24.90878398]
Cp error:  871.71307546

In [111]:


if(save_results and index_error_bis!=-1):
    tu.save_error_list_to_file(errors_linear_dynamic_cp_local[index_error_bis], saving_file_path[index_file]+"errors_linear_dynamic_cp_4dof")
    tu.save_error_list_to_file(errors_linear_dynamic_cp_exp_local[index_error_bis], saving_file_path[index_file]+"errors_linear_dynamic_cp_experiment_4dof")
    tu.save_tf_list_to_file([Tf_list], saving_file_path[index_file]+"TF_list_linear_dynamic_4dof")
    

1.0


In [14]:
rf = importlib.reload(rf)

print("File: ",i)
for speed in l:
    print("Speed: ", speed)
    dist_error,_,errors_prism,_, errors_exp, TF_list = rf.one_inter_prism_resection_advanced(Inter_distance=k, 
                            file_name=i, file_name_marker=j, RF=RF, robot=robot, rate=10, prior = "A", velocity_outlier = speed, 
                            threshold_training = 0.75, number_iteration = 1, threshold_marker=1, min_6dof=True)
    if(len(dist_error)>1):
        RF1 = TF_list[0][1]
        RF2 = TF_list[0][2]
        r1 = R.from_matrix(RF1[0:3,0:3])
        r2 = R.from_matrix(RF2[0:3,0:3])
        euler1 = r1.as_euler('xyz', degrees=True)
        euler2 = r2.as_euler('xyz', degrees=True)
        print(RF1[0:3,3],RF2[0:3,3])
        print(euler1,euler2)
        r1 = R.from_euler('xyz', [0,0,euler1[2]], degrees=True)
        r2 = R.from_euler('xyz', [0,0,euler2[2]], degrees=True)
        RF1[0:3,0:3] = r1.as_matrix()
        RF2[0:3,0:3] = r2.as_matrix()
        RF = [RF1,RF2]
        print("Cp error: ", np.median(errors_prism), np.std(errors_prism))
        print("Inter-prism error: ", np.median(errors_exp), np.std(errors_exp))

File:  ./data/prediction/TS/20220910/10/filtered/f-2-1-1-1-6-1-L_
Speed:  0.01
Selected points:  41
`xtol` termination condition is satisfied.
Function evaluations 489, initial cost 2.9683e-06, final cost 6.2730e-08, first-order optimality 1.73e-10.
Time [s]:  0.5296986103057861
Results done !
[-0.50225547  0.74673403  0.43068446] [0.7822943  0.49453057 0.21896185]
[-1.10846210e-01 -6.31100424e-01 -1.58584766e+02] [ -0.57941273   0.29474274 -62.61634029]
Cp error:  176.6350284512542 178.66316981413794
Inter-prism error:  53.62932781187835 100.544491103535
Speed:  0.05
Selected points:  48
`xtol` termination condition is satisfied.
Function evaluations 188, initial cost 6.7561e-06, final cost 1.5731e-08, first-order optimality 4.73e-10.
Time [s]:  0.2564690113067627
Results done !
[-0.60646799  0.79181431  0.50085601] [0.6867145  0.53185389 0.21330027]
[ 8.53995825e-03  1.49617166e-02 -1.58421861e+02] [-2.94996404e-02 -2.06493331e-02 -6.24531665e+01]
Cp error:  24.096642950743977 30.239

In [ ]:
%matplotlib notebook
tu = importlib.reload(tu)

index_file = 0
path = linear_inter_prism_file_name[index_file]

trimble_1 = tu.read_prediction_data_resection_csv_file(path+"1.csv")
trimble_2 = tu.read_prediction_data_resection_csv_file(path+"2.csv")
trimble_3 = tu.read_prediction_data_resection_csv_file(path+"3.csv")

In [ ]:
print(len(np.array(trimble_1)))
print(len(np.array(trimble_2)))
print(len(np.array(trimble_3)))

In [ ]:
tu = importlib.reload(tu)
rate = 10
velocity_outlier_list = 0.01
index1 = tu.find_not_moving_points_GP(np.array(trimble_1), velocity_outlier_list, 1/rate) 
index2 = tu.find_not_moving_points_GP(np.array(trimble_2), velocity_outlier_list, 1/rate) 
index3 = tu.find_not_moving_points_GP(np.array(trimble_3), velocity_outlier_list, 1/rate)  

In [ ]:
#p1s_l = np.array(trimble_1)[:,1:5]
#p2s_l = np.array(trimble_2)[:,1:5]
#p3s_l = np.array(trimble_3)[:,1:5]
p1s_l = np.array(trimble_1)[index1,1:5]
p2s_l = np.array(trimble_2)[index1,1:5]
p3s_l = np.array(trimble_3)[index1,1:5]
print(len(np.array(trimble_1).T[0]),len(np.array(trimble_2).T[0]),len(np.array(trimble_3).T[0]))
print(len(p1s_l),len(p2s_l),len(p3s_l))

In [ ]:
former_point = [0,0,0]
index_keep = []
count = 0
for i in p1s_l:
    if(np.linalg.norm(i[0:3]-former_point)>0.01):
        index_keep.append(count)
        former_point = i[0:3]
    count+=1
print(len(index_keep))

In [ ]:
%matplotlib notebook

plt.figure(figsize=(10,3))
plt.scatter(p1s_l[index_keep,0],p1s_l[index_keep,2])
plt.scatter(p2s_l[index_keep,0],p2s_l[index_keep,2])
plt.scatter(p3s_l[index_keep,0],p3s_l[index_keep,2])
#plt.plot(p2s_l[:,2])

plt.show()

In [ ]:
import time

Distan_prism = Inter_distance[index_file]

f = lambda x: cost_fun(p1s_l, p2s_l, p3s_l, x[:6], x[6:], Dist_prism_12, Dist_prism_13, Dist_prism_23)

# start_time = time.time()
# res = scipy.optimize.minimize(lambda x: cost_fun(p1s_l, p2s_l, p3s_l, x[:6], x[6:], Dist_prism_12, Dist_prism_13, 
#                             Dist_prism_23), x0=np.zeros((12,1)), tol=1e-8)

# stop_time = time.time()
# print(stop_time - start_time)
# print(res.fun)
# xi_12 = res.x[:6]
# xi_13 = res.x[6:] 
# T12 = exp_T(xi_12)
# T13 = exp_T(xi_13)

In [ ]:
print(len(p1s_l))
print(res.fun/len(p1s_l))

In [ ]:
p1s_r = p1s_l
p2s_r = T12@p2s_l.T
p3s_r = T13@p3s_l.T

#p1s_r = np.array(trimble_1)[:,1:5]
#p2s_r = T12@np.array(trimble_2)[:,1:5].T
#p3s_r = T13@np.array(trimble_3)[:,1:5].T

print(T12)
print(T13)

In [ ]:
%matplotlib notebook

p2s_rarr = np.array(p2s_r).T
p3s_rarr = np.array(p3s_r).T

plt.figure(figsize=(10,3))
#plt.scatter(p1s_r[:,0],p1s_r[:,1], color='b')
#plt.scatter(p2s_rarr[:,0],p2s_rarr[:,2], color='r')
#plt.scatter(p3s_rarr[:,1],p3s_rarr[:,2], color='g')
plt.plot(p2s_rarr[:,2], color='g')
plt.show()

In [ ]:
plt.figure(figsize=(10,3))
offset_timestamp = min(np.array(trimble_1)[0,0],np.array(trimble_2)[0,0],np.array(trimble_3)[0,0])
#plt.plot(np.array(trimble_1)[:,0]-offset_timestamp,p1s_r[:,0])
plt.plot(np.array(trimble_2)[:,0]-offset_timestamp,p2s_rarr[:,0])
#plt.plot(np.array(trimble_3)[:,0]-offset_timestamp,p3s_rarr[:,0])
plt.show()

In [ ]:
plt.figure(figsize=(10,3))
#plt.plot(np.array(trimble_1)[:,0]-offset_timestamp,p1s_r[:,1])
plt.plot(np.array(trimble_2)[:,0]-offset_timestamp,p2s_rarr[:,1])
#plt.plot(np.array(trimble_3)[:,0]-offset_timestamp,p3s_rarr[:,1])
plt.show()

In [ ]:
plt.figure(figsize=(10,3))
#plt.plot(np.array(trimble_1)[:,0]-offset_timestamp,p1s_r[:,2])
plt.plot(np.array(trimble_2)[:,0]-offset_timestamp,p2s_rarr[:,2])
#plt.plot(np.array(trimble_3)[:,0]-offset_timestamp,p3s_rarr[:,2])
plt.show()

In [ ]:
dist_prism = []
timestamp = np.array(trimble_1)[:,0]
for i in range(0,len(timestamp)-1):
    dp1 = abs(np.linalg.norm(p1s_r[i,0:3]-p2s_rarr[i,0:3])-Dist_prism_12)*1000
    dp2 = abs(np.linalg.norm(p1s_r[i,0:3]-p3s_rarr[i,0:3])-Dist_prism_13)*1000
    dp3 = abs(np.linalg.norm(p3s_rarr[i,0:3]-p2s_rarr[i,0:3])-Dist_prism_23)*1000
    dist_prism.append(np.array([timestamp[i], dp1, dp2, dp3]))
dist_prism = np.array(dist_prism)

In [ ]:
print("Mean inter-prism [mm]: ", round(np.mean(dist_prism[:,1]),2), round(np.mean(dist_prism[:,2]),2), round(np.mean(dist_prism[:,3]),2))
print("Std inter-prism [mm]: ", round(np.std(dist_prism[:,1]),2), round(np.std(dist_prism[:,2]),2), round(np.std(dist_prism[:,3]),3))
plt.figure(figsize=(10,5))
offset_timestamp = min(dist_prism[:,0])
plt.subplot(111)
plt.plot(dist_prism[:,0]-offset_timestamp, dist_prism[:,1], label="P12", color="red", alpha=0.5)
plt.plot(dist_prism[:,0]-offset_timestamp, dist_prism[:,2], label="P13", color="blue", alpha=0.5)
plt.plot(dist_prism[:,0]-offset_timestamp, dist_prism[:,3], label="P23", color="green", alpha=0.5)
ax = plt.gca()
ax.set_yscale('symlog')
plt.legend(loc='best')
ax.set_ylabel("Error [mm]")
limitx = [-50, 270]
ax.set_xlim(limitx)
ax.set_xlabel("Time [s] \n Mean inter-prism [mm]: " + str(round(np.mean(dist_prism[:,1]),2)) + "  " + str(round(np.mean(dist_prism[:,2]),2)) + "  " + str(round(np.mean(dist_prism[:,3]),2))
             + "\n Std inter-prism [mm]: " + str(round(np.std(dist_prism[:,1]),2)) + "  " + str(round(np.std(dist_prism[:,2]),2)) + "  " + str(round(np.std(dist_prism[:,3]),2)))
plt.savefig("./figs/20220505_cones_dynamic_L-1-6.jpg")
#plt.savefig("./figs/20220505_cones_dynamic.jpg")
#plt.savefig("./figs/20220523_empty.jpg")
plt.show()  

In [ ]:
# Convert point data to csv file
ttfu = importlib.reload(ttfu)

# Trajectory total station (time, point data) x3
ttfu.Convert_datap_to_csv(np.array(trimble_1)[:,0], np.array(p1s_r), "./data/prediction/nr_trimble_1.csv")
ttfu.Convert_datap_to_csv(np.array(trimble_2)[:,0], np.array(p2s_rarr), "./data/prediction/nr_trimble_2.csv")
ttfu.Convert_datap_to_csv(np.array(trimble_3)[:,0], np.array(p3s_rarr), "./data/prediction/nr_trimble_3.csv")

In [ ]:
ttfu = importlib.reload(ttfu)
save_tf = True
name_file = './data/calibration/tf_automated_resection/20220505_empty_tf_a.csv'
if(save_tf):
    ttfu.save_tf(np.identity(4), T12, T13, name_file)